In [2]:
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_absolute_error

# === 1. Cargar dataset ===
df = pd.read_csv("dstn_5.csv")
df['periodo'] = pd.to_datetime(df['periodo'])

# 🔧 Limpiar nombres de columnas
df.columns = df.columns.str.replace(r"[^\w]", "_", regex=True)

# === 2. Features y target ===
features = [col for col in df.columns if col not in ['product_id', 'periodo', 'tn']]
target = 'tn'

# === 3. Separar train y validación ===
df_train = df[df['periodo'] < '2019-12-01']
df_val = df[df['periodo'] == '2019-12-01']

X_train = df_train[features].copy()
y_train = df_train[target]
X_val = df_val[features].copy()
y_val = df_val[target]

# 🔁 4. Convertir columnas object a category ===
for col in X_train.select_dtypes(include='object').columns:
    X_train[col] = X_train[col].astype('category')
    X_val[col] = X_val[col].astype('category')

# === 5. Función objetivo de Optuna ===
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 800),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'num_leaves': trial.suggest_int('num_leaves', 20, 256),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 200),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 3.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 3.0),
        'random_state': 42
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return mean_absolute_error(y_val, y_pred)

# === 6. Ejecutar la búsqueda bayesiana con persistencia ===
study = optuna.create_study(
    direction="minimize",
    study_name="lgbm_dstn51",
    storage="sqlite:///optuna_lgbm_dstn51.db",
    load_if_exists=True
)
study.optimize(objective, n_trials=50)

# === 7. Ver resultados ===
print("✅ Mejor MAE:", study.best_value)
print("🧪 Mejores hiperparámetros:", study.best_params)


[I 2025-06-30 09:51:37,543] A new study created in RDB with name: lgbm_dstn51


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012619 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22015
[LightGBM] [Info] Number of data points in the train set: 23660, number of used features: 108
[LightGBM] [Info] Start training from score 4.641575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

[I 2025-06-30 09:51:40,365] Trial 0 finished with value: 0.14091777837997563 and parameters: {'n_estimators': 359, 'learning_rate': 0.18756431547363134, 'max_depth': 4, 'num_leaves': 161, 'subsample': 0.9938103988858971, 'colsample_bytree': 0.9077265846002558, 'min_data_in_leaf': 100, 'reg_alpha': 2.2870664532195546, 'reg_lambda': 1.3870589151539958}. Best is trial 0 with value: 0.14091777837997563.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-06-30 09:51:41,420] Trial 1 finished with value: 0.12222168012459421 and parameters: {'n_estimators': 243, 'learning_rate': 0.08348650480290606, 'max_depth': 10, 'num_leaves': 197, 'subsample': 0.8303781495393843, 'colsample_bytree': 0.9219011686591732, 'min_data_in_leaf': 157, 'reg_alpha': 2.5403133779069487, 'reg_lambda': 0.14347777086687663}. Best is trial 1 with value: 0.12222168012459421.


...

[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=55
[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=55
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013867 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22015
[LightGBM] [Info] Number of data points in the train set: 23660, number of used features: 108
[LightGBM] [Info] Start training from score 4.641575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

[I 2025-06-30 09:54:21,503] Trial 48 finished with value: 0.10089684229346355 and parameters: {'n_estimators': 629, 'learning_rate': 0.13654972132258078, 'max_depth': 11, 'num_leaves': 69, 'subsample': 0.7802384903752363, 'colsample_bytree': 0.9908466905759405, 'min_data_in_leaf': 55, 'reg_alpha': 2.6719054537424616, 'reg_lambda': 2.944308269400525}. Best is trial 40 with value: 0.09930277264817379.


[LightGBM] [Warning] min_data_in_leaf is set=54, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=54
[LightGBM] [Warning] min_data_in_leaf is set=54, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=54
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036442 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22015
[LightGBM] [Info] Number of data points in the train set: 23660, number of used features: 108
[LightGBM] [Info] Start training from score 4.641575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

[I 2025-06-30 09:54:31,953] Trial 49 finished with value: 0.10690233430762723 and parameters: {'n_estimators': 653, 'learning_rate': 0.1969569079236772, 'max_depth': 11, 'num_leaves': 66, 'subsample': 0.7694193222978642, 'colsample_bytree': 0.995041925616014, 'min_data_in_leaf': 54, 'reg_alpha': 2.999581850546674, 'reg_lambda': 2.999129257401555}. Best is trial 40 with value: 0.09930277264817379.


[LightGBM] [Warning] min_data_in_leaf is set=54, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=54
✅ Mejor MAE: 0.09930277264817379
🧪 Mejores hiperparámetros: {'n_estimators': 261, 'learning_rate': 0.14224983178543588, 'max_depth': 10, 'num_leaves': 23, 'subsample': 0.8093211391867557, 'colsample_bytree': 0.9231869885459679, 'min_data_in_leaf': 29, 'reg_alpha': 2.699481657936518, 'reg_lambda': 2.8632226095063658}
